In [16]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


In [17]:
# Load the dictionary from the pickle file
with open('signs_dict6.pkl', 'rb') as f:
    data = pkl.load(f)

In [18]:
X_train = np.array(data['X_train'])
X_test = np.array(data['X_test'])
y_train = np.array(data['y_train'])
y_test = np.array(data['y_test'])

In [19]:
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(3120, 4)
(3120, 200, 150, 3)
(53, 4)
(53, 200, 150, 3)


In [24]:
# Define the model
model = keras.Sequential([
    layers.Input(shape=(150, 200, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Add dropout for regularization
    layers.Dense(4, activation='softmax')  # 4 classes
])

In [25]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 148, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 148, 198, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 74, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 72, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 72, 97, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 36, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 34, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 34, 46, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 17, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 15, 21, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 15, 21, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 7, 10, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 17920)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │     4,587,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,979,140 (18.99 MB)

 Trainable params: 4,978,180 (18.99 MB)

 Non-trainable params: 960 (3.75 KB)

In [26]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Train the model with data augmentation
history = model.fit(X_train, y_train, 
                    batch_size=32, 
                    epochs=50, 
                    validation_data=(X_test, y_test), 
                    callbacks=[keras.callbacks.EarlyStopping(patience=9, restore_best_weights=True, start_from_epoch=40), 
                               ])

Epoch 1/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 260ms/step - accuracy: 0.2970 - loss: 13.8137 - val_accuracy: 0.2453 - val_loss: 7.8616
Epoch 2/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 257ms/step - accuracy: 0.3258 - loss: 12.3982 - val_accuracy: 0.3774 - val_loss: 3.0928
Epoch 3/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 253ms/step - accuracy: 0.3267 - loss: 6.7951 - val_accuracy: 0.4717 - val_loss: 2.4604
Epoch 4/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 252ms/step - accuracy: 0.3730 - loss: 3.4223 - val_accuracy: 0.3208 - val_loss: 1.4366
Epoch 5/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.3784 - loss: 1.6908 - val_accuracy: 0.4906 - val_loss: 1.2490
Epoch 6/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.3920 - loss: 1.3801 - val_accuracy: 0.4717 - val_loss: 1.5358
Epoch 7/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 252ms/step - accuracy: 0.4232 - loss: 1.2551 - val_accuracy: 0.5094 - val_loss: 1.2493
Epoch 8/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 254ms/step - accuracy: 0.4195 - loss: 1.2505 - val_ac

In [ ]:
# Print classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=['crosswalk', 'speedlimit', 'stop', 'trafficlight']))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
              precision    recall  f1-score   support

   crosswalk       0.54      0.74      0.62        19
  speedlimit       0.56      0.62      0.59        32
        stop       0.60      0.53      0.56        17
trafficlight       0.25      0.12      0.16        17

    accuracy                           0.53        85
   macro avg       0.49      0.50      0.48        85
weighted avg       0.50      0.53      0.51        85



In [ ]:
# Save the model to a file
model.save('traffic_signs_model5.keras')